In [1]:
from config import  functions
from dotenv import load_dotenv

import os

load_dotenv()

True

In [2]:
import numpy as np 
import random 
def get_function_params(function_detail):
    function_name = function_detail["name"]
    function_parameters = function_detail["parameters"]["properties"]
    required_parameters = function_detail["parameters"].get("required", [])
    optional_parameters = function_detail["parameters"].get("optional", [])
    
    function_args_choices_req = {}
    
    for param in required_parameters:
        param_details = function_parameters[param]
        param_type = param_details["type"]
        function_args_choices_req[param] = {}
        if 'enum' in param_details:
            function_args_choices_req[param]['values'] = param_details['enum']
            function_args_choices_req[param]['type'] = param_type
        else:
            if param_type == "boolean":
                function_args_choices_req[param]['values'] = [True, False]
                function_args_choices_req[param]['type'] = param_type
            elif param_type in ["integer", "number"]:
                lower_bound = param_details.get("lower_bound", 1)
                upper_bound = param_details.get("upper_bound", 100)
                # print(function_args_choices_req)
                # print(function_name, lower_bound, upper_bound,param_details)
                function_args_choices_req[param]['values'] = list(range(lower_bound, upper_bound))
                function_args_choices_req[param]['type'] = param_type
            elif param_type == "string":
                function_args_choices_req[param]['values'] = ["|string1|", "|string2|", "|string3|"]
                function_args_choices_req[param]['type'] = param_type
        
    function_args_choices_opt = {}
    for param in optional_parameters:
        function_args_choices_opt[param] = {}
        param_details = function_parameters[param]
        param_type = param_details["type"]
        if 'enum' in param_details:
            function_args_choices_opt[param]['values'] = param_details['enum']
            function_args_choices_opt[param]['type'] = param_type
        else:
            if param_type == "boolean":
                function_args_choices_opt[param]['values'] = [True, False]
                function_args_choices_opt[param]['type'] = param_type
            elif param_type in ["integer", "number"]:
                lower_bound = param_details.get("lower_bound", 1)
                upper_bound = param_details.get("upper_bound", 100)
                # print(function_name, lower_bound, upper_bound,param_details)
                function_args_choices_opt[param]['values'] = list(range(lower_bound, upper_bound))
                function_args_choices_opt[param]['type'] = param_type
            elif param_type == "string":
                function_args_choices_opt[param]['values'] = ["|string1|", "|string2|", "|string3|"]
                function_args_choices_opt[param]['type'] = param_type

    return function_name, function_args_choices_req, function_args_choices_opt

In [11]:
def construct_function(required_params_config, opt_params_config, opt_prob=0.5):
    function_args = {}
    for param, choices in required_params_config.items():
        if 'array' not in choices['type']:
            function_args[param] = random.choice(choices['values'])
        else:
            # choose a random length for the array based on the number of choices
            array_length = random.randint(1, len(choices['values']))
            # choose random values for the array based on the choices with no duplicates
            function_args[param] = random.sample(choices['values'], array_length)
            if 'all' in function_args[param] and len(function_args[param]) > 1:
                function_args[param].remove('all')
                
    
    for param, choices in opt_params_config.items():
        if random.random() < opt_prob:
            # print(param, choices)
            if 'array' not in choices['type']:
                # print(f"Optional param {param} is not an array")
                function_args[param] = random.choice(choices['values'])
            else:
                # print(f"Optional param {param} is an array")
                # choose a random length for the array based on the number of choices
                array_length = random.randint(1, len(choices['values']))
                # print(f"Array length: {array_length}")
                # choose random values for the array based on the choices with no duplicates
                function_args[param] = random.sample(choices['values'], array_length)
                
                if 'all' in function_args[param] and len(function_args[param]) > 1:
                    function_args[param].remove('all')
                
                # print(f"Array values: {function_args[param]}")
    
    return function_args


def generate_function_call_args(function_details, n=10, opt_prob=0.5, function_args={}):
    
    for function_detail in function_details:
        function_name, required_params_config, opt_params_config = get_function_params(function_detail)
        function_args[function_name] = []
        # print(function_name, required_params_config, opt_params_config)
        for _ in range(n):
            args = construct_function(required_params_config, opt_params_config, opt_prob=opt_prob)
            function_args[function_name].append(args)
    
    return function_args


calling_params = generate_function_call_args(functions, n=6, opt_prob=0.5) ## Generate 10 function calls for each function

## Assimilate all function calls into a readable and maintainable format

function_call_main_dict = {}
for function_name, function_args in calling_params.items():
    function_call_main_dict[function_name] = {"args": [], "calls" : []}
    function_call_main_dict[function_name]['args'].append(function_args)
    for args in function_args:
        function_call = function_name + "("
        for param, value in args.items():
            function_call += f"{param}={value}, "
        function_call = function_call[:-2] + ")" if len(args) > 0 else function_call + ")"
        function_call_main_dict[function_name]['calls'].append(function_call)


In [12]:
import json 
with open("./data/function_calls.json", "w") as f:
    json.dump(function_call_main_dict, f)

In [13]:
# read the saved function calls
with open("./data/function_calls.json", "r") as f:
    function_calls = json.load(f)